In [1]:
import sys
import json
from collections import defaultdict
import numpy as np

# vseek imports
sys.path.append("../")
import vseek.common.vseek_paths as vsp
from vseek.common.loader import load_genes_metadata, load_genome, load_viral_genes
from vseek.utils.sequence_io import SequenceIO
from vseek.utils.sra_callers import download_fasta
from vseek.common.io_files import get_viral_genome_fasta_paths, get_genome_genes_paths, get_meta_genomes_paths
from vseek.utils.vseek_analysis import dynamic_hamming


In [2]:
# loading all datasets 
viral_genome_paths = get_viral_genome_fasta_paths()
viral_genes_paths = get_genome_genes_paths()
all_accessions = list(viral_genome_paths.keys())

In [3]:
# downloading metagenome files 
# takes 5 minutes on 60MBPS network
srr_id = "SRR12464727"
download_fasta(srr_id)

Prefetching SRR12464727 data...


'/home/axiomcura/Development/prelim/VSeek/results/fasta_files'

In [4]:
# since it is one file, it is returning a str 
# if it is multiple files, it will return a list
metagenome_path = get_meta_genomes_paths()
reader = SequenceIO(metagenome_path)

In [18]:
# parameters 
all_accessions = list(viral_genome_paths.keys())
metagenome_path = get_meta_genomes_paths()
reader = SequenceIO(metagenome_path)
reads = reader.lazy_load_fasta()
threshold=0.40 # threshold between 40% and 80% is enough to get genius level 

counts = defaultdict(lambda: 0)
for idx, read in enumerate(reads):

    if idx % 5 == 0:
        # saving every 5 analyzed reads
        with open("./viral_composition_counts.json", "w") as outfile:
            json.dump(counts, outfile)

    if idx % 100 == 0:
        print(f"Currently on read number {idx}")

    read_score = {}
    for acc_id in all_accessions:
        # load all the gene sequences
        gene_sequences = load_viral_genes(acc_id)

        top_score = 0.0
        for gene in gene_sequences:
            score = dynamic_hamming(read=read.sequence, reference=gene)
            if score >= threshold:
                if score == 1:
                    top_score = score
                    break
                elif score > top_score: 
                    top_score = score

        read_score[acc_id] = top_score

    top_score_acc_ids = max(read_score, key=read_score.get)
    counts[top_score_acc_ids] += 1


with open("./viral_composition_counts.json", "w") as outfile:
    json.dump(counts, outfile)

Currently on read number 0
Currently on read number 100
Currently on read number 200
Currently on read number 300
Currently on read number 400
Currently on read number 500
Currently on read number 600
Currently on read number 700
Currently on read number 800
Currently on read number 900
Currently on read number 1000
Currently on read number 1100
Currently on read number 1200
Currently on read number 1300
Currently on read number 1400
Currently on read number 1500
Currently on read number 1600
Currently on read number 1700
Currently on read number 1800
Currently on read number 1900
Currently on read number 2000
Currently on read number 2100
Currently on read number 2200
Currently on read number 2300


KeyboardInterrupt: 

In [8]:
d = {"Hello": 7, "world": 10, "smile": 15}
max_value_key = max(d, key=d.get)
d[max_value_key]

15

In [14]:
counts

defaultdict(<function __main__.<lambda>()>, {'NC_006273': 2, 'NC_001664': 1})